## Description of the steps

### 1- Rosters (gameDate, playerId, teamId)
* statusCode: 11 values
* status : 16 values

### 2- Games (gamePk, gameDate)
since some games are postponed or arrested, we have to choose the last line of the game
* gameType : 8
* codedGameState : 6
* detailedGameState : 7
* isTie : 2
* seriesDescription : 8

### 3-playerBoxScores (playerId, gameDate,gamePk)
* jerseyNum: 100
* positionCode: 12
* positionName: 8
* positionType: 6


### 4-teamBoxScores (teamId, index,gamePk)
* No categorical variable

### 5- events

### 6- transactions

### 7- awards

### 8- standings

### 9- playerTwitterFollowers

### 10- teamTwiterFollowers
Ok

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df
#

In [ ]:
#==============ROSTERS========================================
statusCode = {'A':1, 'D60':2, 'RM':3, 'D10':4, 'D7':5, 'PL':6, 'SU':7, 
              'FME':8, 'BRV':9, 'RES':10,'DEC':11}
status = {'Active':1, 'Injured 60-Day':2, 'Reassigned to Minors':3,
       'Injured 10-Day':4, 'Injured 7-Day':5, 'Paternity List':6,
       'Suspended # days':7, 'Family Medical Emergency':8, 'Bereavement List':9,
       'Reserve List (Minors)':10, 'Deceased':11, '60-day IL':12, '10-day IL':13,
       'Reassigned':14, 'Paternity':15, 'Suspended':16}
#============== GAME =================================#
gameType = {'E':1, 'S':2, 'R':3, 'A':4, 'F':5, 'D':6, 'L':7, 'W':8}
codedGameState = {'F':1, 'C':2, 'D':3, 'S':4, 'U':5, 'O':6}
detailedGameState = {'Final':1, 'Cancelled':2, 'Completed Early':3, 'Postponed':4, 
                     'Scheduled':5,'Suspended':6, 'Game Over':7}
isTie = {False:1, True:2}
seriesDescription = {'Exhibition':1, 'Spring Training':2, 'Regular Season':3, 'All-Star Game':4,
                     'Wild Card Game':5, 'Division Series':6, 'League Championship Series':7,
                     'World Series':8}
#===============================PLAYER SCORES============================
positionName = {'Catcher':1,'Designated Hitter':2,'First Base':3,'Outfielder':4,
                'Pinch Hitter':5,'Pinch Runner':6,'Pitcher':7, 'Second Base':8,
                'Shortstop':9,'Third Base':10}
positionType = {'Catcher':1, 'Hitter':2, 'Infielder':3, 'Outfielder':4, 'Pitcher':5, 'Runner':6}
#=================================GAMES========================
doubleHeader = {"N":1, "Y":2, "S":3}
dayNight = {"day":1, "night":2}
#=================================================

In [ ]:
SCORES = ['home', 'flyOuts', 'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns', 'strikeOuts', 
          'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch', 'atBats', 'caughtStealing', 'stolenBases', 
          'groundIntoDoublePlay', 'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi', 'leftOnBase', 
          'sacBunts', 'sacFlies', 'catchersInterference', 'pickoffs', 'airOutsPitching', 'groundOutsPitching', 
          'runsPitching', 'doublesPitching', 'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching', 
          'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching', 'hitByPitchPitching', 'atBatsPitching', 
          'caughtStealingPitching', 'stolenBasesPitching', 'inningsPitched', 'earnedRuns', 'battersFaced', 'outsPitching', 
          'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching', 'rbiPitching', 'inheritedRunners', 'inheritedRunnersScored', 
          'catchersInterferencePitching', 'sacBuntsPitching', 'sacFliesPitching']
ren_player = {col:col+"_p" for col in SCORES}
ren_team = {col:col+"_t" for col in SCORES}

In [ ]:
time_index = "index"
#============================
rosters_id = [time_index, 'playerId']
rosters_cat = ["statusCode","status"]
#=========================
pscores_id = ["playerId","index", 'teamId'] #, "gamePk"
pscores_cat = ["jerseyNum","positionCode","positionName","positionType"]
pscores_num = ['home', 
               'gamesPlayedBatting', 'flyOuts', 'groundOuts', 'runsScored', 'doubles', 'triples', 
               'homeRuns', 'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch', 
               'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay', 'groundIntoTriplePlay', 
               'plateAppearances', 'totalBases', 'rbi', 'leftOnBase', 'sacBunts', 'sacFlies', 
               'catchersInterference', 'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching', 
               'completeGamesPitching', 'shutoutsPitching', 'winsPitching', 'lossesPitching', 'flyOutsPitching', 
               'airOutsPitching', 'groundOutsPitching', 
               'runsPitching', 'doublesPitching', 'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching', 
               'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching', 'hitByPitchPitching', 
               'atBatsPitching', 'caughtStealingPitching', 'stolenBasesPitching', 'inningsPitched', 
               'saveOpportunities', 'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 
               'balls', 'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching', 'rbiPitching', 
               'gamesFinishedPitching', 'inheritedRunners', 'inheritedRunnersScored', 'catchersInterferencePitching', 
               'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves', 'assists', 
               'putOuts', 'errors', 'chances']
pscores_num = [col+'_p' if (col in SCORES) else col for col in pscores_num]
#==============
tscores_id = ["teamId", "index","gamePk"]
tscores_num = ['home', 'flyOuts',
       'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
       'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
       'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
       'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
       'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
       'pickoffs', 'airOutsPitching', 'groundOutsPitching', 'runsPitching',
       'doublesPitching', 'triplesPitching', 'homeRunsPitching',
       'strikeOutsPitching', 'baseOnBallsPitching', 'intentionalWalksPitching',
       'hitsPitching', 'hitByPitchPitching', 'atBatsPitching',
       'caughtStealingPitching', 'stolenBasesPitching', 'inningsPitched',
       'earnedRuns', 'battersFaced', 'outsPitching', 'hitBatsmen', 'balks',
       'wildPitches', 'pickoffsPitching', 'rbiPitching', 'inheritedRunners',
       'inheritedRunnersScored', 'catchersInterferencePitching',
       'sacBuntsPitching', 'sacFliesPitching']
tscores_num = [col+'_t' if (col in SCORES) else col for col in tscores_num]
#===========================
games_cat = ["gameType","codedGameState","detailedGameState","isTie","seriesDescription",
             "doubleHeader", "dayNight"]
games_id = ['gamePk','homeId', 'awayId']
games_num = ['gameNumber',  'scheduledInnings', 'gamesInSeries', 'homeWins', 'homeLosses', 
             'homeWinPct', 'homeWinner', 'homeScore', 'awayWins', 'awayLosses', 'awayWinPct', 
             'awayWinner', 'awayScore']
# season (cat) / 'resumeDate', 'resumedFrom' (continuity)
#===========================

In [ ]:
#============================
def process_rosters(df):
    df["statusCode"] = df["statusCode"].map(statusCode)
    df["status"] = df["status"].map(status)
    return df
#=================
def process_pscores(df):
    # deal with battingOrder
    df["positionName"] = df["positionName"].map(positionName)
    df["positionType"] = df["positionType"].map(positionType)
    df["jerseyNum"] = df["jerseyNum"].fillna(100)
    df["positionCode"] = df["positionCode"].fillna(0)
    return df
#======================================
def process_games(df):
    #add season
    df["gameType"] = df["gameType"].map(gameType)
    df["codedGameState"] = df["codedGameState"].map(codedGameState)
    df["detailedGameState"] = df["detailedGameState"].map(detailedGameState)
    df["isTie"] = df["isTie"].map(isTie)
    df["seriesDescription"] = df["seriesDescription"].map(seriesDescription)
    df["doubleHeader"] = df["doubleHeader"].map(doubleHeader)
    df["dayNight"] = df["dayNight"].map(dayNight)
    return df
#====================================

#========================

In [ ]:
import pandas as pd
import numpy as np
import gc
import os
from tqdm import tqdm

In [ ]:
DATA_ROOT = "../input/mlb-pdef-train-dataset"

In [ ]:
#os.listdir(DATA_ROOT)

In [ ]:
DEBUG = True

In [ ]:
%%time
tgt = pd.read_csv(f"{DATA_ROOT}/nextDayPlayerEngagement_train.csv")
tgt.drop("engagementMetricsDate", axis=1, inplace=True)
tgt["date"] = pd.to_datetime(tgt["date"], format='%Y%m%d')
tgt = tgt.rename(columns={"date":"EvalDate"})

if DEBUG:
    print(tgt.shape)
    tgt = tgt.loc[tgt[time_index]>=1000].reset_index(drop=True)
    print(tgt.shape)
#
gc.collect()

In [ ]:
tgt.head(2)

In [ ]:
tgt["index"].max()

In [ ]:
%%time
rosters = pd.read_csv(f"{DATA_ROOT}/rosters_train.csv", usecols=rosters_cat + rosters_id)
rosters = process_rosters(rosters)

In [ ]:
%%time
pscores = pd.read_csv(f"{DATA_ROOT}/playerBoxScores_train.csv")
pscores = pscores.rename(columns=ren_player)
pscores = pscores[pscores_id+pscores_cat+pscores_num]
pscores = process_pscores(pscores)
#================
NO_DOUBLON = True
if NO_DOUBLON:
    print(pscores.shape)
    pscores = pscores.drop_duplicates(subset=rosters_id, keep="last")
    print(pscores.shape)

In [ ]:
tscores = pd.read_csv(f"{DATA_ROOT}/teamBoxScores_train.csv")
tscores = tscores.rename(columns=ren_team)
tscores = tscores[tscores_id + tscores_num]

In [ ]:
print(tscores.shape)
tscores  = tscores.drop_duplicates(subset=["teamId", "index"], keep="last")
print(tscores.shape)

In [ ]:
games = pd.read_csv(f"{DATA_ROOT}/games_train.csv", usecols=games_id + games_cat + games_num)
games = process_games(games)
games = games.drop_duplicates(subset="gamePk", keep="last")

## BUILD DATASET...

In [ ]:
%%time
#======
tgt = tgt.merge(rosters, on=["playerId","index"], how="left")
tgt = tgt.merge(pscores, on=["playerId", "index"],  how="left")
tgt = tgt.merge(tscores, on=["teamId", "index"],  how="left")#, suffixes=["_p", "_t"]
tgt = tgt.merge(games, on="gamePk", how="left")
print(tgt.shape)
#====
gc.collect()

## REDUCING MEMORY USAGE

In [ ]:
IDCOLS = list( set( rosters_id + pscores_id + tscores_id + games_id ) ) 
IDCOLS = IDCOLS + ["EvalDate"]
CATCOLS = rosters_cat + pscores_cat  + games_cat
TGTCOLS = ["target1","target2","target3","target4"]
NUMCOLS = pscores_num + tscores_num + games_num 

In [ ]:
TOFLOAT = ["awayWinPct", "homeWinPct"]
TOINT = [col for col in NUMCOLS + CATCOLS if col not in TOFLOAT]

In [ ]:
%%time
def reduce_df(df, train=True):
    df["jerseyNum"] = df["jerseyNum"].fillna(100) #100
    # positionCode #12
    df[NUMCOLS + CATCOLS] = df[NUMCOLS + CATCOLS].fillna(0.)
    df[TOINT] = df[TOINT].astype(int)
    if train:
        lab = df[TGTCOLS].copy()
        df.drop(TGTCOLS, axis=1, inplace=True)
        df = reduce_mem_usage(df)
        df = df.join(lab)
    else:
        df = reduce_mem_usage(df)
    gc.collect()
    return df

In [ ]:
%%time
tgt  = reduce_df(tgt, train=True)

In [ ]:
tgt.info()

In [ ]:
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [ ]:
import tensorflow as tf
import tensorflow.keras.layers as L
import tensorflow.keras.models as M
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

In [ ]:
def make_model(n_in):
    inp = L.Input(name="inputs", shape=(n_in,))
    nh = 50
    x = L.Dense(nh, activation="relu", name="d1")(inp)
    x = L.Dense(nh, activation="relu", name="d2")(x)
    #x = L.Dense(nh, activation="relu", name="d3")(x)
    preds = L.Dense(4, activation="linear", name="preds")(x)
    
    model = M.Model(inp, preds, name="ANN")
    model.compile(loss="mean_absolute_error", optimizer="adam")
    return model

In [ ]:
X = tgt[NUMCOLS].values
y = tgt[TGTCOLS].values

In [ ]:
%%time
sc  = StandardScaler()
X = sc.fit_transform(X)
gc.collect()

In [ ]:
net = make_model(X.shape[1])
print(net.summary())

In [ ]:
X.mean()

In [ ]:
net.fit(X, y, batch_size=30_000, epochs=10)
gc.collect()

## INFERENCE

In [ ]:
null = np.nan
true = True
false = False

In [ ]:
START_DATE = pd.to_datetime("2018-01-01")

In [ ]:
#LAST_DF = tgt.loc[tgt[time_index]>850].copy().reset_index(drop=True)
#PLAYERS = pd.DataFrame(tgt.playerId.unique(), columns=["playerId"])
#LAST_DF.shape, tgt.shape

In [ ]:
import mlb
env = mlb.make_env() # initialize the environment
iter_test = env.iter_test() # iterator which loops over each date in test se

In [ ]:
"""
FE, SUB = [], []
for (test_df, sample_prediction_df) in iter_test: 
    FE.append(test_df)
    SUB.append(sample_prediction_df)
    env.predict(sample_prediction_df)
#===========
"""

In [ ]:
def prepare_test(hid_df, hid_sub):
    hid_tgt = hid_sub[["date","playerId"]].copy()
    #================== PART ONE ========================#
    # adding rosters
    colname = "rosters"
    """#consider input as null
    rosters = pd.DataFrame(columns=rosters_cat + rosters_id)
    rosters["playerId"] = hid_tgt["playerId"]
    """
    if hid_df[colname].iloc[0] ==  hid_df[colname].iloc[0]:
        rosters = pd.DataFrame(eval(hid_df[colname].iloc[0]))
        rosters["index"] = (pd.to_datetime(rosters["gameDate"]) - START_DATE).dt.days
        rosters = rosters[rosters_cat + rosters_id]
    else:
        rosters = pd.DataFrame(columns=rosters_cat + rosters_id)
        rosters["playerId"] = hid_tgt["playerId"]
    rosters = process_rosters(rosters)
    # adding player scores
    colname = "playerBoxScores"
    #consider input as null
    pscores = pd.DataFrame(columns = pscores_id + pscores_cat + pscores_num)
    pscores = pscores.rename(columns=ren_player)
    pscores["playerId"] = hid_tgt["playerId"]
    """
    if hid_df[colname].iloc[0] ==  hid_df[colname].iloc[0]:## An error occurs in this block.
        pscores = pd.DataFrame(eval(hid_df[colname].iloc[0]))
        pscores = pscores.rename(columns=ren_player)
        pscores["index"] = (pd.to_datetime(pscores["gameDate"]) - START_DATE).dt.days
        pscores = pscores[pscores_id + pscores_cat + pscores_num]
    else:
        pscores = pd.DataFrame(columns = pscores_id + pscores_cat + pscores_num)
        pscores = pscores.rename(columns=ren_player)
        pscores["playerId"] = hid_tgt["playerId"]
    """
    pscores = process_pscores(pscores)
    pscores = pscores.drop_duplicates(subset="playerId", keep="last")
    # Merging
    hid_tgt = hid_tgt.merge(rosters, on="playerId", how="left")
    hid_tgt = hid_tgt.merge(pscores, on="playerId", how="left")
    #===================PART TWO=============================
    colname = "teamBoxScores"
    """#consider input as null
    tscores = pd.DataFrame(columns = tscores_id  + tscores_num)
    tscores = tscores.rename(columns=ren_team)
    tscores["teamId"] = list(hid_tgt["teamId"].unique())
    """
    if hid_df[colname].iloc[0] ==  hid_df[colname].iloc[0]:
        tscores = pd.DataFrame(eval(hid_df[colname].iloc[0]))
        tscores = tscores.rename(columns=ren_team)
        tscores["index"] = (pd.to_datetime(tscores["gameDate"]) - START_DATE).dt.days
        tscores = tscores[tscores_id  + tscores_num]
    else:
        tscores = pd.DataFrame(columns = tscores_id  + tscores_num)
        tscores = tscores.rename(columns=ren_team)
        tscores["teamId"] = list(hid_tgt["teamId"].unique())
        
    tscores  = tscores.drop_duplicates(subset="teamId", keep="last")
    hid_tgt = hid_tgt.merge(tscores, on="teamId", how="left")
    #==================PART THREE ============================
    colname = "games"
    """#consider input as null
    games = pd.DataFrame(columns = games_id  + games_cat + games_num )
    games["gamePk"] = list(hid_tgt["gamePk"].unique())
    """
    if hid_df[colname].iloc[0] ==  hid_df[colname].iloc[0]:
        games = pd.DataFrame(eval(hid_df[colname].iloc[0]))
        games["index"] = (pd.to_datetime(games["gameDate"]) - START_DATE).dt.days
        games = games[games_id  + games_cat + games_num]
    else:
        games = pd.DataFrame(columns = games_id  + games_cat + games_num )
        games["gamePk"] = list(hid_tgt["gamePk"].unique())
    games = process_games(games)
    games = games.drop_duplicates(subset="gamePk", keep="last")
    hid_tgt = hid_tgt.merge(games, on="gamePk", how="left")
    #================================================
    hid_tgt  = reduce_df(hid_tgt, train=False)
    return hid_tgt

In [ ]:
"""
idx = 4
hid_df = FE[idx].copy()
hid_sub = SUB[idx].copy()
print(hid_sub.shape[0])
"""

In [ ]:
"""
hid_sub = hid_sub.reset_index()
hid_sub["playerId"] = hid_sub["date_playerId"].apply(lambda x: int( x.split("_")[1] ))
hid_sub.drop(TGTCOLS, axis=1, inplace=True)

hid_tgt = prepare_test(hid_df, hid_sub)
Xe = hid_tgt[NUMCOLS].values
Xe = sc.transform(Xe)
    
pe = net.predict(Xe)
pe = np.clip(pe, 0, 100)
    
hid_sub[TGTCOLS] = pe
sub = hid_sub[["date_playerId"] + TGTCOLS].copy()
"""

In [ ]:

for (hid_df, hid_sub) in iter_test: 
    hid_sub = hid_sub.reset_index()
    hid_sub["playerId"] = hid_sub["date_playerId"].apply(lambda x: int( x.split("_")[1] ))
    hid_sub.drop(TGTCOLS, axis=1, inplace=True)
    
    hid_tgt = prepare_test(hid_df, hid_sub)
    assert hid_tgt.shape[0] == hid_sub.shape[0]
    Xe = hid_tgt[NUMCOLS].values
    Xe = sc.transform(Xe)
    
    #pe = net.predict(Xe)
    #pe = np.clip(pe, 0, 100)
    
    hid_sub[TGTCOLS] = 0.
    sub = hid_sub[["date_playerId"] + TGTCOLS].copy()
    sub = sub.fillna(0.)
    env.predict(sub)
    gc.collect()
#===========

In [ ]:
sub.tail()